# Problem statement

* Input:
    - $X \in \{0, 1\}^{n\times f}$ binary matrix of participants (rows) and their attributes (columns).
    - $k < n$ integer number of desired participants.  
    - $q \in [0, 1]^f$ target Bernoulli distribution for each feature
    - $w \in \mathbb{R}_+^f$ weighting over features
 
* Goal: select a $k$-hot vector $y \in \{0, 1\}^n$ with $\|y\| = k$ such that the KL divergence between $D(E[\langle y, X\rangle ] ~\|~ q)$ is minimized.



---
### Notes

* $D(p~\|~q) := \sum_i p_i \log \frac{p_i}{q_i}$
* This seems np-hard in general, but probably approximable via submodular optimization

### Web app

* Build this in a nice little flask app with a CSV uploader, and bootstrap widgets to control $k$ and $q$
* Visualize the distribution of the solution

### Implementation

* Let $v_i$ be the indicator vector of selected rows at step $i$ of a greedy algorithm.
* The distribution for the $j$th attribute is 
$$
p_j = \frac{\langle y, X \rangle_j}{\|y\|}
$$

* The (weighted) objective function is:

$$
f(y) = - \sum_j w_j D(p_j \| q_j)
$$
where
$$
D(a \| b) = a\log \frac{a}{b} + (1-a)\log\frac{1-a}{1-b}
$$

We maximize the objective by iteratively optimizing marginal improvement:

$$
y_{t+1} \leftarrow y_t + \text{arg}\max_e f(y_t + e) - f(y_t)
$$

$$
p'_j = \frac{\langle y, X \rangle_j + \langle e, X \rangle_j }{1 + \|y\|}
= \frac{\langle y, X \rangle_j + X_{ej} }{1 + \|y\|}
= \frac{\|y\| p_j + X_{ej}}{1 + \|y\|}
$$

$$
\Delta f = - \sum_j w_j \left(D(p'_j \| q_j) - D( p_j \| q_j) \right)
$$

### Additional ideas

* Random initialization: run $T$ threads in parallel with random seeds; pick the one that achieves the best objective.
* Alternately, initialiaze by the pair of items with the minimum overlap (equivalently, maximum weighted l2 score).
* Or just use pre-selects and try to compensate with remaining selectors.

In [254]:
import numpy as np
import scipy
import numba

In [175]:
def obj(p, w, q):
    # Prevent numerical underflow in log
    
    amin = 1e-200
    
    pbar = 1. - p
    qbar = 1. - q
    
    H = p * (np.log(p + amin) - np.log(q + amin)) + pbar * (np.log(pbar + amin) - np.log(qbar + amin))
    
    return - H.dot(w)

def _entrofy(X, k, w=None, q=None, pre_selects=None):
    '''See entrofy() for documentation'''

    n_participants, n_attributes = X.shape

    if w is None:
        w = np.ones(n_attributes)

    if q is None:
        q = 0.5 * np.ones(n_attributes)

    assert 0 < k <= n_participants
    assert not np.any(w < 0)
    assert np.all(q >= 0.0) and np.all(q <= 1.0)
    assert len(w) == n_attributes
    assert len(q) == n_attributes

    if k == n_participants:
        return np.arange(n_participants)

    # Initialization
    y = np.zeros(n_participants, dtype=bool)

    if pre_selects is None:
        # Select one at random
        pre_selects = np.random.choice(n_participants, size=1)

    y[pre_selects] = True

    # Where do we have missing data?
    Xn = np.isnan(X)

    while True:
        i = y.sum()
        if i >= k:
            break

        # Initialize the distribution vector
        p = np.nanmean(X[y], axis=0)
        p[np.isnan(p)] = 0.0

        # Compute the candidate distributions
        p_new = (p * i + X) / (i + 1.0)

        # Wherever X is nan, propagate the old p since we have no new information
        p_new[Xn] = (Xn * p)[Xn]

        # Compute marginal gain for each candidate
        delta = obj(p_new, w, q) - obj(p, w, q)

        # Knock out the points we've already taken
        delta[y] = -np.inf

        # Select the top score.  Break near-ties randomly.
        target_score = delta.max()
        target_score = target_score - 1e-3 * np.abs(target_score)
        new_idx = np.random.choice(np.flatnonzero(delta >= target_score))
        y[new_idx] = True

    return obj(np.nanmean(X[y], axis=0), w, q), np.flatnonzero(y)


def entrofy(X, k, w=None, q=None, pre_selects=None, n_samples=15):
    '''Entrofy your panel.
    
    Parameters
    ----------
    X : np.ndarray, shape=(n, f), dtype=bool
        Rows are participants
        Columns are attributes
        
    k : int in (0, n]
        The number of participants to select
        
    w : np.ndarray, non-negative, shape=(f,)
        Weighting over the attributes
        By default, a uniform weighting is used
        
    q : np.darray, values in [0, 1], shape=(f,)
        Target distribution vector for the attributes.
        By default, 1/2
        
    pre_selects : None or iterable
        Optionally, you may pre-specify a set of rows to be forced into the solution.
        
    n_samples : int > 0
        If pre_selects is None, run `n_samples` random initializations and return
        the solution with the best objective value.
        
        
    Returns
    -------
    score : float
        The score of the solution found.  Larger is better.
        
    idx : np.ndarray, shape=(k,)
        Indicies of the selected rows
        
    '''
    if pre_selects is not None:
        n_samples = 1
        
    results = [_entrofy(X, k, w=w, q=q, pre_selects=pre_selects) for i in range(n_samples)]
    
    max_score, best = results[0]
    for score, solution in results[1:]:
        if score > max_score:
            max_score = score
            best = solution
            
    return max_score, best

In [176]:
X = np.random.randn(300, 5)
X = np.greater_equal(X, np.linspace(-1.5, 1.5, X.shape[1]))

In [177]:
X.shape

(300, 5)

In [178]:
score, y = entrofy(X, 10, n_samples=5)

In [179]:
score, y

(-0.040271027101377699,
 array([  0,   3,  79,  95, 112, 148, 168, 217, 259, 270]))

In [180]:
score, y

(-0.040271027101377699,
 array([  0,   3,  79,  95, 112, 148, 168, 217, 259, 270]))

In [181]:
X[y].mean(axis=0)

array([ 0.6,  0.5,  0.5,  0.5,  0.4])

In [182]:
y

array([  0,   3,  79,  95, 112, 148, 168, 217, 259, 270])

In [183]:
X[y]

array([[False,  True, False, False,  True],
       [ True,  True,  True,  True, False],
       [ True, False,  True, False,  True],
       [ True, False,  True, False,  True],
       [False, False,  True,  True, False],
       [ True,  True, False, False, False],
       [ True, False, False,  True, False],
       [False, False, False, False, False],
       [False,  True, False,  True, False],
       [ True,  True,  True,  True,  True]], dtype=bool)

---

In [184]:
import pandas as pd
import numpy as np

In [115]:
def binarize(df, n_bins=5):
    
    df2 = pd.DataFrame(index=df.index)
    
    for column in df:
        # If it's a float, chop up into bins
        if np.issubdtype(df[column].dtype, float):
            data = pd.cut(df[column], n_bins)
        else:
            data = df[column]
            
        # If it's categorical or object, do this
        unique_values = data.unique()
        for value in unique_values:
            if value is np.nan:
                continue
            new_name = name='{}__{}'.format(column, value)
            
            new_series = pd.DataFrame(data=(data == value), dtype=float)
            if not np.any(new_series):
                continue
            df2[new_name] = new_series
            df2[new_name][pd.isnull(data)] = np.nan
            
        
    return df2

In [109]:
df = pd.read_csv('/home/bmcfee/Desktop/list-300.csv', skipinitialspace=True, index_col=0)

In [110]:
ages = 30 + 5 * np.random.randn(len(df))
ages[np.random.choice(len(ages), size=len(ages)//10)] = np.nan

df['age'] = ages

In [111]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 300 entries, Michael Trotter to Edward Baum
Data columns (total 5 columns):
gender          298 non-null object
career_stage    266 non-null object
country         299 non-null object
subfield        266 non-null object
age             271 non-null float64
dtypes: float64(1), object(4)
memory usage: 14.1+ KB


In [112]:
df.head(10)

,gender,career_stage,country,subfield,age
name,,,,,
Michael Trotter,male,senior-faculty,OECD,cosmology,29.795674
Eunice Mcginnis,female,junior-faculty,OECD,NaN,29.294910
Denise Dobbs,female,junior-faculty,OECD,cosmology,17.728281
Dorothy Hitch,female,senior-faculty,OECD,cosmology,25.453735
Donnell Nicole,male,grad-student,OECD,astrophysics,28.320383
Patricia Krueger,female,NaN,OECD,astrophysics,31.352296
Bernice Wade,female,grad-student,OECD,NaN,26.479273
Ann Valdez,female,junior-faculty,OECD,exoplanets,28.305690
Robert Campbell,male,junior-faculty,non-OECD,astrophysics,26.098247


In [113]:
df['age'].dtype

dtype('float64')

In [116]:
df_bin = binarize(df)

In [120]:
df_bin.reset_index()

,name,gender__male,gender__female,career_stage__senior-faculty,career_stage__junior-faculty,career_stage__grad-student,country__OECD,country__non-OECD,subfield__cosmology,subfield__astrophysics,subfield__exoplanets,subfield__solar,"age__(25.785, 31.163]","age__(15.00117, 20.406]","age__(20.406, 25.785]","age__(31.163, 36.542]","age__(36.542, 41.92]"
0,Michael Trotter,1,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0
1,Eunice Mcginnis,0,1,0,1,0,1,0,NaN,NaN,NaN,NaN,1,0,0,0,0
2,Denise Dobbs,0,1,0,1,0,1,0,1,0,0,0,0,1,0,0,0
3,Dorothy Hitch,0,1,1,0,0,1,0,1,0,0,0,0,0,1,0,0
4,Donnell Nicole,1,0,0,0,1,1,0,0,1,0,0,1,0,0,0,0
5,Patricia Krueger,0,1,NaN,NaN,NaN,1,0,0,1,0,0,0,0,0,1,0
6,Bernice Wade,0,1,0,0,1,1,0,NaN,NaN,NaN,NaN,1,0,0,0,0
7,Ann Valdez,0,1,0,1,0,1,0,0,0,1,0,1,0,0,0,0
8,Robert Campbell,1,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0
9,Nicholas Langley,1,0,0,1,0,1,0,NaN,NaN,NaN,NaN,0,0,0,1,0
